# Ejercicio 40
Para los contenidos geolocalizados: ¿Cuál es el contenido más cercano del que fue editado más recientemente?¿Y la diferencia entre sus tiempos de edición?

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from datetime import datetime

In [2]:
df_geotags = pd.read_csv('wikipedia_dataset-2c2021/geo_tags.csv')

In [3]:
df_geotags

,gt_id,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,158041,4328020,earth,0,41.507500,-5.817500,1000,NaN,NaN,NaN,NaN
1,5844377,4670424,earth,0,13.738000,-89.292000,10000,city,NaN,NaN,NaN
2,5844381,4670424,earth,0,13.702000,-89.208000,10000,city,NaN,NaN,NaN
3,5844383,4670424,earth,0,13.400000,-89.750000,10000,city,NaN,NaN,NaN
4,5844388,4670424,earth,0,12.850000,-88.000000,10000,city,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
556388,136672179,5239183,earth,0,3.450000,-76.533333,1000,NaN,NaN,NaN,NaN
556389,136672180,104468,earth,1,30.815839,45.996069,1000,NaN,NaN,NaN,NaN
556390,136672181,968281,earth,1,46.953333,6.256944,10000,city,NaN,NaN,NaN
556391,136672182,79985,earth,0,45.109659,7.641221,500,NaN,NaN,NaN,NaN


In [4]:
df_contents = pd.read_csv('wikipedia_dataset-2c2021/contents.csv')

In [5]:
df_contents

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,NaN,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...
...,...,...,...,...,...,...,...,...,...,...
4132159,Cruz nestoriana,10010593,0,138052016,138051910.0,2021-09-01T12:06:00Z,Copydays,474907.0,NaN,termino traducción y quito plantilla en desarr...
4132160,Manifestación del orgullo gay de 1977 en Barce...,10010595,0,138051859,NaN,2021-09-01T11:52:28Z,Crls9011,4032712.0,NaN,Crls9011 trasladó la página [[Manifestación de...
4132161,Rocasaurus,10010596,0,138051902,NaN,2021-09-01T11:57:06Z,Gabriel Solo,158577.0,NaN,Gabriel Solo trasladó la página [[Rocasaurus]]...
4132162,Medalla Conmemorativa del 50.º Aniversario de ...,10010598,0,138051961,NaN,2021-09-01T12:01:38Z,Términus,5429951.0,NaN,Términus trasladó la página [[Medalla Conmemor...


In [6]:
df_contents['revision_timestamp']

0          2021-01-19T18:40:34Z
1          2021-08-31T18:27:01Z
2          2021-08-28T17:17:54Z
3          2021-08-29T17:16:00Z
4          2020-11-12T19:56:15Z
                   ...         
4132159    2021-09-01T12:06:00Z
4132160    2021-09-01T11:52:28Z
4132161    2021-09-01T11:57:06Z
4132162    2021-09-01T12:01:38Z
4132163    2021-09-01T12:03:14Z
Name: revision_timestamp, Length: 4132164, dtype: object

In [7]:
df_contents['revision_timestamp'].hasnans

False

In [8]:
df_contents.loc[:,'revision_timestamp'] = pd.to_datetime(df_contents['revision_timestamp'])
df_contents['revision_timestamp']

0         2021-01-19 18:40:34+00:00
1         2021-08-31 18:27:01+00:00
2         2021-08-28 17:17:54+00:00
3         2021-08-29 17:16:00+00:00
4         2020-11-12 19:56:15+00:00
                     ...           
4132159   2021-09-01 12:06:00+00:00
4132160   2021-09-01 11:52:28+00:00
4132161   2021-09-01 11:57:06+00:00
4132162   2021-09-01 12:01:38+00:00
4132163   2021-09-01 12:03:14+00:00
Name: revision_timestamp, Length: 4132164, dtype: datetime64[ns, UTC]

In [9]:
df_contents['revision_timestamp'].describe()

<ipython-input-9-19776d067a90>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_contents['revision_timestamp'].describe()


count                       4132164
unique                      4006833
top       2021-02-20 09:59:51+00:00
freq                            166
first     2002-10-23 09:34:16+00:00
last      2021-09-01 15:17:32+00:00
Name: revision_timestamp, dtype: object

In [10]:
merged_df = pd.merge(left=df_contents,right=df_geotags,left_on='id',right_on='gt_page_id',how='inner')

In [11]:
merged_df.shape

(537305, 21)

In [12]:
merged_df

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,...,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,Andorra,7,0,138035057,138027475.0,2021-08-31 18:27:01+00:00,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido,...,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
1,Argentina,10,0,137966826,137825165.0,2021-08-28 17:17:54+00:00,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,...,10,earth,0,-34.599722,-58.381944,10000,city,NaN,NaN,NaN
2,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29 17:16:00+00:00,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,...,15,earth,1,42.506111,1.522222,1000,landmark,NaN,NaN,NaN
3,Anoeta,26,0,133531913,125347732.0,2021-02-25 18:09:01+00:00,InternetArchiveBot,4704851.0,NaN,Rescatando 1 referencia(s) y marcando 0 enlace...,...,26,earth,1,43.162222,-2.070000,10000,city,NaN,NaN,NaN
4,Afganistán,39,0,138047554,138043235.0,2021-09-01 05:01:16+00:00,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...,...,39,earth,0,34.532778,69.165833,10000,city,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537300,Kumarghat,10010550,0,138051050,138051042.0,2021-09-01 10:28:24+00:00,Carlukas,86975.0,NaN,Añadiendo la [[Categoría:Localidades de Tripur...,...,10010550,earth,1,24.158333,92.029722,10000,city,NaN,NaN,NaN
537301,Río Makarora,10010559,0,138051210,138051186.0,2021-09-01 10:45:37+00:00,Lorito987,5952489.0,NaN,NaN,...,10010559,earth,1,-44.316667,169.166667,1000,landmark,NaN,NaN,NaN
537302,Panisagar,10010567,0,138051314,138051311.0,2021-09-01 10:58:51+00:00,Carlukas,86975.0,NaN,Añadiendo la [[Categoría:Localidades de Tripur...,...,10010567,earth,1,24.250000,92.140000,10000,city,NaN,NaN,NaN
537303,Dewanpasa,10010581,0,138051603,138051596.0,2021-09-01 11:30:24+00:00,Carlukas,86975.0,NaN,Añadiendo la [[Categoría:Localidades de Tripur...,...,10010581,earth,1,24.360639,92.139850,10000,city,NaN,NaN,NaN


In [13]:
merged_df = merged_df.sort_values('revision_timestamp',ascending=False)
merged_df

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,...,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
38365,Cacabelos,650097,0,138053815,138053805.0,2021-09-01 14:02:20+00:00,Arbaborix,5172488.0,NaN,NaN,...,650097,earth,1,42.599722,-6.725556,10000,city,NaN,NaN,NaN
533503,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,41.934278,-5.660000,1000,NaN,NaN,NaN,NaN
533530,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.270389,-5.858917,1000,NaN,NaN,NaN,NaN
533525,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.421778,-5.782528,1000,NaN,NaN,NaN,NaN
533526,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.384556,-5.780083,1000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14571,Wikipedia:Café/Octubre de 2005,211478,4,11665014,5002793.0,2007-09-28 17:10:23+00:00,Muro Bot,341614.0,NaN,Bot: Cambiando nombre a plantilla trasladada,...,211478,earth,0,4.698333,-74.148056,1000,NaN,NaN,NaN,NaN
14570,Wikipedia:Café/Octubre de 2005,211478,4,11665014,5002793.0,2007-09-28 17:10:23+00:00,Muro Bot,341614.0,NaN,Bot: Cambiando nombre a plantilla trasladada,...,211478,earth,0,40.416667,-3.750000,1000,NaN,NaN,NaN,NaN
14568,Wikipedia:Café/Octubre de 2005 01,211477,4,11665003,2784818.0,2007-09-28 17:10:07+00:00,Muro Bot,341614.0,NaN,Bot: Cambiando nombre a plantilla trasladada,...,211477,earth,0,4.698333,-74.148056,1000,NaN,NaN,NaN,NaN
14569,Wikipedia:Café/Octubre de 2005 01,211477,4,11665003,2784818.0,2007-09-28 17:10:07+00:00,Muro Bot,341614.0,NaN,Bot: Cambiando nombre a plantilla trasladada,...,211477,earth,0,20.500000,-144.900000,1000,NaN,NaN,NaN,NaN


In [14]:
merged_df=merged_df.reset_index(drop=True)

In [15]:
merged_df.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,...,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,Cacabelos,650097,0,138053815,138053805.0,2021-09-01 14:02:20+00:00,Arbaborix,5172488.0,NaN,NaN,...,650097,earth,1,42.599722,-6.725556,10000,city,NaN,NaN,NaN
1,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,41.934278,-5.660000,1000,NaN,NaN,NaN,NaN
2,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.270389,-5.858917,1000,NaN,NaN,NaN,NaN
3,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.421778,-5.782528,1000,NaN,NaN,NaN,NaN
4,Anexo:Estaciones de la línea Plasencia-Astorga,9946348,104,138053764,138039951.0,2021-09-01 13:58:58+00:00,ListeriaBot,3918773.0,NaN,Wikidata list updated [V2],...,9946348,earth,0,40.384556,-5.780083,1000,NaN,NaN,NaN,NaN


In [16]:
last_revision = merged_df.iloc[[0],:]
last_revision

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,...,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,Cacabelos,650097,0,138053815,138053805.0,2021-09-01 14:02:20+00:00,Arbaborix,5172488.0,NaN,NaN,...,650097,earth,1,42.599722,-6.725556,10000,city,NaN,NaN,NaN


In [17]:
first_geo_tag=merged_df[['gt_lat','gt_lon']].values[0]
first_geo_tag

array([42.59972222, -6.72555556])

In [18]:
merged_df=merged_df.drop([0])

In [19]:
buscador = NearestNeighbors()
buscador.fit(merged_df[['gt_lat','gt_lon']].values)

NearestNeighbors()

In [20]:
dist, index = buscador.kneighbors([[42.59972222, -6.72555556]],n_neighbors=1)

In [21]:
index.flatten()

array([129181], dtype=int64)

In [22]:
nearest_revision = merged_df.iloc[index.flatten(),:]
nearest_revision=nearest_revision.reset_index(drop=True)
nearest_revision

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment,...,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,Batalla de Cacabelos,6102755,0,134763096,130486504.0,2021-04-14 18:33:45+00:00,NaN,NaN,79.148.201.153,/* Batalla */,...,6102755,earth,1,42.599722,-6.725556,5000,event,NaN,NaN,NaN


In [23]:
last_revision_timestamp = last_revision.loc[0,'revision_timestamp']
last_revision_timestamp

Timestamp('2021-09-01 14:02:20+0000', tz='UTC')

In [24]:
nearest_revision_timestamp = nearest_revision.loc[0,'revision_timestamp']
nearest_revision_timestamp

Timestamp('2021-04-14 18:33:45+0000', tz='UTC')

In [25]:
time_difference=last_revision_timestamp-nearest_revision_timestamp

## Resultado

In [26]:
print('The nearest location from', last_revision.loc[0,'title'],'is',nearest_revision.loc[0,'title'],', and their difference betweeen their latest modifications are:',time_difference)

The nearest location from Cacabelos is Batalla de Cacabelos , and their difference betweeen their latest modifications are: 139 days 19:28:35
